In [ ]:
import os
import sys
import json
import shutil
from multiprocessing import Process

In [ ]:
col_key = 'UF'
col_value = 'VALOR PARCELA'
sep = ';'

In [ ]:
dir = os.path.dirname(os.getcwd()) + '/'
sys.path.append(dir)

In [ ]:
from src import reader,mappers

In [ ]:
csv, index_col_key, index_col_value = reader.csv(path=f'{dir}data/202101_BolsaFamilia_Pagamentos.csv',col_key='UF',col_value='VALOR PARCELA',sep=';')

In [ ]:
number_mappers = int(len(csv)/1000000)
number_mappers += 1 if len(csv)%1000000 != 0 else 0

million = 1000000

In [ ]:
os.mkdir('temp')
list_mappers = []
for i_mapper in range(number_mappers):
    if i_mapper == 0:
        list_mappers.append(Process(target=mappers.run,args=(csv[0:million],index_col_key,index_col_value,i_mapper)))
    elif i_mapper == number_mappers-1:
        list_mappers.append(Process(target=mappers.run,args=(csv[million*i_mapper:len(csv)],index_col_key,index_col_value,i_mapper)))
    else:
        list_mappers.append(Process(target=mappers.run,args=(csv[million*i_mapper:million*(i_mapper+1)],index_col_key,index_col_value,i_mapper)))

for mapper in list_mappers:
    mapper.start()

for mapper in list_mappers:
    mapper.join()

In [ ]:
list_json_files = os.listdir('temp')
aggreagate_date = {}
for file in list_json_files:
    temp_data = json.loads(open(f'temp/{file}','r').read())

    keys = set(aggreagate_date.keys()).union(temp_data.keys())
    aggreagate_date = {key: aggreagate_date.get(key, 0) + temp_data.get(key, 0) for key in keys}

In [ ]:
aggreagate_date = {key: aggreagate_date[key] for key in sorted(aggreagate_date)}

In [ ]:
shutil.rmtree('temp')
aggreagate_date

In [ ]:
with open(f'{dir}data/aggregate_data.json','w') as output_file:
    json.dump(aggreagate_date,output_file)